In [1]:
import pandas as pd
import json

Importing the data into the corresponding data frames

In [3]:
df_brands = pd.read_json('E:/ASU/PhD/Interviews/Fetch_AE/Data/brands/brands.json', lines=True)
df_receipts = pd.read_json('E:/ASU/PhD/Interviews/Fetch_AE/Data/receipts/receipts.json', lines=True)
df_users = pd.read_json('E:/ASU/PhD/Interviews/Fetch_AE/Data/users/users.json', lines=True)


Helper methods for data cleaning:
The primary key and the date columns in the data are stored as dictionaries with key values. These methods help to extract them into the corresponding columns.

In [4]:
def extract_oid(value):
        if isinstance(value, dict) and '$oid' in value:
            return value['$oid']
        else:
            return None
        
def extract_cpg_oid(value):
        if isinstance(value, dict) and '$id' in value and '$oid' in value['$id']:
            return value['$id']['$oid']
        else:
            return None
        
def convert_date(value):
        if isinstance(value, dict) and '$date' in value:
            timestamp_ms = value['$date']
            return pd.to_datetime(timestamp_ms, unit='ms')
        else:
            return None

In [5]:

df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


Extract CPG table from the Brand table and have cpg_id as a foreign key in the brand table.

In [33]:
cpg_dict = df_brands['cpg']
cpg = []
for dic in cpg_dict:
    if '$id' in dic and '$ref' in dic:
        id_val = dic['$id']['$oid']
        ref_val = dic['$ref']
        cpg.append({'id': id_val, 'ref': ref_val})

df_cpg = pd.DataFrame(cpg)

df_cpg


,id,ref
0,601ac114be37ce2ead437550,Cogs
1,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead437559,Cogs
4,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...
1162,5f77274dbe37ce6b592e90bf,Cogs
1163,53e10d6368abd3c7065097cc,Cogs
1164,5332fa12e4b03c9a25efd1e7,Cogs
1165,5332f5f6e4b03c9a25efd0b4,Cogs


In [34]:
df_brands['_id'] = df_brands['_id'].apply(extract_oid)
df_brands['cpg'] = df_brands['cpg'].apply(extract_cpg_oid)
df_brands.rename(columns={'_id':'brand_id', 'cpg':'cpg_id'}, inplace=True)

df_brands

,brand_id,barcode,category,categoryCode,cpg_id,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,5f77274dbe37ce6b592e90bf,test brand @1601644365844,NaN,NaN
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,53e10d6368abd3c7065097cc,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,5332f5f6e4b03c9a25efd0b4,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


In [35]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


All the date columns are checked for consistency.

In [36]:
df_receipts['_id'] = df_receipts['_id'].apply(extract_oid)
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    df_receipts[col] = df_receipts[col].apply(convert_date)
df_receipts.rename(columns={'_id':'receipt_id', 'userId':'user_id'}, inplace=True)
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    df_receipts[col] = pd.to_datetime(df_receipts[col])

df_receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


Creating a new table for RewardsReceiptItemsList from the Receipt table.

In [37]:
new_rows = []

for index, row in df_receipts.iterrows():
    json_array = row['rewardsReceiptItemList']
    if isinstance(json_array, str):
        json_array = json.loads(json_array)

    if isinstance(json_array, list):
        for item in json_array:
            item['receipt_id'] = row['receipt_id'] 
            new_rows.append(item)

df_rewardsReceiptItems = pd.DataFrame(new_rows)
df_rewardsReceiptItems.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Ensure the barcode columns are of the same type
df_rewardsReceiptItems['barcode'] = df_rewardsReceiptItems['barcode'].astype(str)
df_brands['barcode'] = df_brands['barcode'].astype(str)

df_rewardsReceiptItems = pd.merge(
        df_rewardsReceiptItems,
        df_brands[['barcode', 'brand_id']],
        on='barcode',
        how='left',
        suffixes=('', '_brand') # add a suffix to the brand id to make it clear which id it is.
    )

df_rewardsReceiptItems.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,brand_id
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nan,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dropping the rewardsReceiptItemList column from the Receipt table.

In [39]:
df_receipts.drop('rewardsReceiptItemList', axis=1, inplace=True)
df_receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [40]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [41]:
df_users['_id'] = df_users['_id'].apply(extract_oid)
df_users['createdDate'] = df_users['createdDate'].apply(convert_date)
# df_users['lastLogin'].ffill(inplace=True)
df_users['lastLogin'] = df_users['lastLogin'].apply(convert_date)
df_users.rename(columns={'_id':'user_id'}, inplace=True)
date_columns = ['createdDate', 'lastLogin']
for col in date_columns:
    df_users[col] = pd.to_datetime(df_users[col])

df_users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


In [60]:
df_rewardsReceiptItems.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'receipt_id', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId', 'brand_id'],
      dtype='object')

Queries that directly answer predetermined questions from a business stakeholder

In [64]:
# Question: What are the top 5 brands by receipts scanned for most recent month?

from pandasql import sqldf
import datetime

current_date = datetime.date.today()
current_month = current_date.month
current_year = current_date.year

query_1 = f"""
    SELECT rri.brandCode, COUNT(*) AS brand_count
    FROM df_receipts r
    INNER JOIN df_rewardsReceiptItems rri ON r.receipt_id = rri.receipt_id
    WHERE strftime('%Y-%m', r.dateScanned) = (
        SELECT strftime('%Y-%m', MAX(dateScanned))
        FROM df_receipts)
    GROUP BY rri.brandCode
    ORDER BY brand_count DESC
    LIMIT 5;
    """

print (sqldf(query_1, locals()))

  brandCode  brand_count
0      None           26


In the most recent calendar month from the data, the brandCode is None for all the entries. So, inorder to fetch the top 5 brands from the last 28 days (February 2021 has only 28 days), I slightly modify the query to fetch data from 30 days before the last scanned date. So, by executing this query, the top brands I extracted are BRAND, MISSION, and VIVA.

In [112]:
query_1 = """
    SELECT rri.brandCode, COUNT(*) AS brand_count
    FROM df_receipts r
    INNER JOIN df_rewardsReceiptItems rri ON r.receipt_id = rri.receipt_id
    WHERE strftime('%Y-%m-%d %H:%M:%S', dateScanned) >= strftime('%Y-%m-%d %H:%M:%S', 
                    date((SELECT MAX(dateScanned) FROM df_receipts), '-28 days'))
    GROUP BY rri.brandCode
    ORDER BY brand_count DESC
    LIMIT 5;
"""

print(sqldf(query_1, locals()))

  brandCode  brand_count
0      None          210
1     BRAND            3
2   MISSION            2
3      VIVA            1


In [71]:
max(df_receipts['dateScanned'])

Timestamp('2021-03-01 23:17:34.772000')

In [68]:
# Question: How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

query_2 = f"""
    SELECT rri.brandCode, COUNT(*) AS brand_count
    FROM df_receipts r
    INNER JOIN df_rewardsReceiptItems rri ON r.receipt_id = rri.receipt_id
    WHERE strftime('%Y-%m', r.dateScanned) = (
    SELECT strftime('%Y-%m', date(MAX(dateScanned), '-1 month'))
    FROM df_receipts)
    GROUP BY rri.brandCode
    ORDER BY brand_count DESC
    LIMIT 5;
    """

print (sqldf(query_2, locals()))

  brandCode  brand_count
0      None          184
1     BRAND            3
2   MISSION            2
3      VIVA            1


This shows the top brands from the previous calendar month based on the number of receipts scanned. But, due to the brandCode being None for all the entries in the last calendar month, I modify the query to extract data from the previous month used in the above query.

In [76]:
query_2 = f"""
    SELECT rri.brandCode, COUNT(*) AS brand_count
    FROM df_receipts r
    INNER JOIN df_rewardsReceiptItems rri ON r.receipt_id = rri.receipt_id
    WHERE strftime('%Y-%m-%d %H:%M:%S', dateScanned) <= strftime('%Y-%m-%d %H:%M:%S', date((SELECT MAX(dateScanned) FROM df_receipts), '-28 days'))
    AND   strftime('%Y-%m-%d %H:%M:%S', dateScanned) >= strftime('%Y-%m-%d %H:%M:%S', date((SELECT MAX(dateScanned) FROM df_receipts), '-62 days'))
    GROUP BY rri.brandCode
    ORDER BY brand_count DESC
    LIMIT 6;
    """

print (sqldf(query_2, locals()))

        brandCode  brand_count
0            None         4061
1          HY-VEE          291
2  BEN AND JERRYS          180
3           PEPSI           93
4          KROGER           89
5         KLEENEX           88


This gives the top 5 brands of the previous month which are Hy-VEE, BEN AND JERRYS, PEPSI, KROGER, AND KLEENEX.

In [183]:
# Question: When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

query_3 = f"""
    SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
    FROM df_receipts
    WHERE rewardsReceiptStatus IN ('FINISHED', 'REJECTED')
    GROUP BY rewardsReceiptStatus;
    """

print (sqldf(query_3, locals()))

  rewardsReceiptStatus  average_spend
0             FINISHED      80.854305
1             REJECTED      23.326056


So, the average spend from the receipts from "ACCEPTED" rewardsReceiptStatus is higher than "REJECTED" status.

In [77]:
# Question: When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

query_4 = f"""
    SELECT rewardsReceiptStatus, SUM(purchasedItemCount) AS total_items_purchased
    FROM df_receipts
    WHERE rewardsReceiptStatus IN ('FINISHED', 'REJECTED')
    GROUP BY rewardsReceiptStatus;
    """

print (sqldf(query_4, locals()))

  rewardsReceiptStatus  total_items_purchased
0             FINISHED                 8184.0
1             REJECTED                  173.0


Comparing the total number of items purchased, the receipts with "ACCEPTED" rewardsReceiptStatus is higher.

In [113]:
# Question: Which brand has the most spend among users who were created within the past 6 months?

query_5 = f"""
    SELECT rri.brandCode, SUM(r.totalSpent) as totalSpent
    FROM df_rewardsReceiptItems rri
    JOIN df_receipts r ON rri.receipt_id = r.receipt_id
    WHERE r.user_id IN (
        SELECT DISTINCT user_id
        FROM df_users
        WHERE createdDate >= (
            SELECT strftime('%Y-%m-%d', MAX(createdDate), '-6 months') FROM df_users
        )
    )
    GROUP BY rri.brandCode
    ORDER BY totalSpent DESC
    LIMIT 2;
    """


print (sqldf(query_5, locals()))

        brandCode  totalSpent
0            None  2930282.42
1  BEN AND JERRYS   197337.68


Since most of the brandcode entries are None, ignoring them, the brand with most spend among the users in the last 6 months is BEN AND JERRYS.

In [109]:
# Question: Which brand has the most transactions among users who were created within the past 6 months?

query_6 = f"""
    SELECT rew.brandCode, COUNT(*) AS transaction_count
    FROM df_rewardsReceiptItems rew
    JOIN df_receipts r ON rew.receipt_id = r.receipt_id
    WHERE r.user_id IN (
        SELECT DISTINCT user_id
        FROM df_users
        WHERE createdDate >= (
            SELECT strftime('%Y-%m-%d', MAX(createdDate), '-6 months') FROM df_users
        )
    )
    GROUP BY rew.brandCode
    ORDER BY transaction_count DESC
    LIMIT 2;
    """

print (sqldf(query_6, locals()))

  brandCode  transaction_count
0      None               2412
1    HY-VEE                291


Since most of the brandcode entries are None, ignoring them, the brand with most number transactions among the users in the last 6 months is HY-VEE.

Evaluating Data Quality:

In [114]:
#Data Completeness

df_brands.isnull().sum()

brand_id          0
barcode           0
category        155
categoryCode    650
cpg_id            0
name              0
topBrand        612
brandCode       234
dtype: int64

In [115]:
df_receipts.isnull().sum()

receipt_id                   0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
user_id                      0
dtype: int64

In [116]:
df_rewardsReceiptItems.isnull().sum()

barcode                                  0
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6133
partnerItemId                            0
preventTargetGapPoints                6590
quantityPurchased                      174
userFlaggedBarcode                    6611
userFlaggedNewItem                    6625
userFlaggedPrice                      6649
userFlaggedQuantity                   6649
receipt_id                               0
needsFetchReviewReason                6729
pointsNotAwardedReason                6608
pointsPayerId                         5674
rewardsGroup                          5217
rewardsProductPartnerId               4672
userFlaggedDescription                6743
originalMetaBriteBarcode              6877
originalMetaBriteDescription          6938
brandCode                             4341
competitorRewardsGroup                6673
discountedI

In [117]:
df_users.isnull().sum()

user_id          0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [118]:
df_cpg.isnull().sum()

id     0
ref    0
dtype: int64

In [120]:
#Duplicate row check

df_brands[df_brands.duplicated()]

,brand_id,barcode,category,categoryCode,cpg_id,name,topBrand,brandCode


In [121]:
df_receipts[df_receipts.duplicated()]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,user_id


In [127]:
df_users[df_users['user_id'].duplicated()]

,user_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN


In [128]:
df_rewardsReceiptItems[df_rewardsReceiptItems.duplicated()]

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,brand_id


In [129]:
df_cpg[df_cpg.duplicated()]

,id,ref
3,601ac142be37ce2ead437559,Cogs
5,601ac142be37ce2ead437559,Cogs
6,601ac142be37ce2ead437559,Cogs
12,559c2234e4b06aca36af13c6,Cogs
14,5332f5fbe4b03c9a25efd0ba,Cogs
...,...,...
1160,559c2234e4b06aca36af13c6,Cogs
1161,5332f709e4b03c9a25efd0f1,Cogs
1163,53e10d6368abd3c7065097cc,Cogs
1164,5332fa12e4b03c9a25efd1e7,Cogs


In [212]:
#Data Consistency

df_rewardsReceiptItems.describe()

,quantityPurchased,userFlaggedQuantity,originalMetaBriteQuantityPurchased
count,6774.000000,299.000000,15.000000
mean,1.385740,1.872910,1.200000
std,1.203805,1.314823,0.414039
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000
max,17.000000,5.000000,2.000000


In [215]:
df_brands.describe()

,topBrand
count,555.000000
mean,0.055856
std,0.229850
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [216]:
df_users.describe()

,createdDate,lastLogin
count,495,433
mean,2020-08-06 01:34:47.878830592,2021-01-23 07:48:00.578216960
min,2014-12-19 14:21:22.381000,2018-05-07 17:23:40.003000
25%,2021-01-04 19:30:17.483500032,2021-01-08 18:14:53.928000
50%,2021-01-13 20:19:38.720999936,2021-01-21 13:57:48.697999872
75%,2021-01-25 17:31:59.408999936,2021-02-03 15:34:11.043000064
max,2021-02-12 14:11:06.240000,2021-03-05 16:52:23.204000


In [217]:
df_receipts.describe()

,bonusPointsEarned,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,totalSpent
count,544.000000,1119,1119,568,1119,537,609.000000,671,635.00000,684.000000
mean,238.893382,2021-01-28 02:09:41.600271616,2021-01-28 02:09:41.600272384,2021-01-19 12:10:05.020589568,2021-01-28 15:14:28.703043840,2021-01-18 05:42:47.657355776,585.962890,2020-12-21 10:48:45.994038784,14.75748,77.796857
min,5.000000,2020-10-30 20:17:59,2020-10-30 20:17:59,2021-01-03 15:24:10,2021-01-03 15:24:10,2020-10-30 20:18:00,0.000000,2017-10-30 00:00:00,0.00000,0.000000
25%,5.000000,2021-01-14 19:13:03.690499840,2021-01-14 19:13:03.690499840,2021-01-08 21:22:42.500000,2021-01-14 21:32:25.500000,2021-01-07 18:15:47,5.000000,2021-01-05 00:00:00,1.00000,1.000000
50%,45.000000,2021-01-29 17:18:22,2021-01-29 17:18:22,2021-01-19 21:13:57.500000,2021-01-29 17:18:47,2021-01-19 20:54:02,150.000000,2021-01-13 15:18:59,2.00000,18.200000
75%,500.000000,2021-02-07 13:20:13.736999936,2021-02-07 13:20:13.736999936,2021-01-27 17:42:13.500000,2021-02-07 13:20:13.736999936,2021-01-27 01:12:24,750.000000,2021-01-26 01:12:58,5.00000,34.960000
max,750.000000,2021-03-01 23:17:34.772000,2021-03-01 23:17:34.772000,2021-02-26 22:36:25,2021-03-01 23:17:34.772000,2021-02-26 22:36:25,10199.800000,2021-03-08 17:37:13,689.00000,4721.950000
std,299.091731,NaN,NaN,NaN,NaN,NaN,1357.166947,NaN,61.13424,347.110349


In [239]:
def check_missing_values(df):
    return df.isnull().sum()

def check_dumped_values(df):
    mask = (df == '') | (df == ' ') | (df == 0)
    return mask.sum()[mask.sum()>0]

def check_consistency(df):
    for col in df.columns:
        if (pd.api.types.is_integer_dtype(df[col])) and (df[col] < 0).any():
            print(f"Negative values count in {col}: {df[df[col] < 0][col].count()}")
    return

def check_date_format(df):
    date_columns = [col for col in df.columns if 'date' in col.lower()]
    if (len(date_columns)==0):
        return
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
        print(f"Column {col} successfully converted to datetime.")
    return

def check_date_range(df):
    date_columns = [col for col in df.columns if 'date' in col.lower()]
    for col in date_columns:
        print(f"Oldest {col}: {df[col].min()}")
        print(f"Latest {col}: {df[col].max()}")
    return

def check_UUID(df):
    uuid_cols = [col for col in df.columns if 'id' in col.lower() or 'barcode' in col.lower()]
    for col in uuid_cols:
       print("Number of Unique IDs in {col}: ", len(df[col].unique()))
    return

def check_duplicate_rows(df):
    print("Duplicate rows:", df.duplicated().sum())
    return


# Missing and Dumped Value Analysis

print("--- df_brands ---")
print(df_brands.info())
print("\nMissing values:\n", check_missing_values(df_brands))
print("\nDumped values:\n", check_dumped_values(df_brands))

print("\n--- df_receipts ---")
print(df_receipts.info())
print("\nMissing values:\n", check_missing_values(df_receipts))
print("\nDumped values:\n", check_dumped_values(df_receipts))

print("\n--- df_rewardsReceiptItems ---")
print(df_rewardsReceiptItems.info())
print("\nMissing values:\n", check_missing_values(df_rewardsReceiptItems))
print("\nDumped values:\n", check_dumped_values(df_rewardsReceiptItems))

print("\n--- df_users ---")
print(df_users.info())
print("\nMissing values:\n", check_missing_values(df_users))
print("\nDumped values:\n", check_dumped_values(df_users))


# Data Type Consistency and Range Checks

# Negative check
print("\n--- df_brands ---")
check_consistency(df_brands)
print("\n--- df_receipts ---")
check_consistency(df_receipts)
print("\n--- df_rewardReceiptItemsList ---")
check_consistency(df_rewardsReceiptItems)
print("\n--- df_users ---")
check_consistency(df_users)

# Check Date Format
print("\n--- df_brands ---")
check_date_format(df_brands)
print("\n--- df_receipts ---")
check_date_format(df_receipts)
print("\n--- df_rewardReceiptItemsList ---")
check_date_format(df_rewardsReceiptItems)
print("\n--- df_users ---")
check_date_format(df_users)

# Check Date Range
print("\n--- df_brands ---")
check_date_range(df_brands)
print("\n--- df_receipts ---")
check_date_range(df_receipts)
print("\n--- df_rewardReceiptItemsList ---")
check_date_range(df_rewardsReceiptItems)
print("\n--- df_users ---")
check_date_range(df_users)

# Unique ID check: Foreign key UUIDs will be redundant. It gives a picture of association relationships between the tables. So checking duplicated for Primary Key
print("\n--- df_brands ---")
check_UUID(df_brands)
print("\n--- Duplicate Primary Key values ---")
check_duplicate_rows(df_brands['brand_id'])

print("\n--- df_receipts ---")
check_UUID(df_receipts)
print("\n--- Duplicate Primary Key values ---")
check_duplicate_rows(df_receipts['receipt_id'])

print("\n--- df_rewardReceiptItemsList ---")
check_UUID(df_rewardsReceiptItems)
print("\n--- Duplicate Primary Key values ---")
check_duplicate_rows(df_rewardsReceiptItems['barcode'])

print("\n--- df_users ---")
check_UUID(df_users)
print("\n--- Duplicate Primary Key values ---")
check_duplicate_rows(df_users['user_id'])


# Duplicate Records
print("\n--- df_brands: Duplicate Records ---")
check_duplicate_rows(df_brands)
print("\n--- df_receipts: Duplicate Records ---")
check_duplicate_rows(df_receipts)
print("\n--- df_rewardsReceiptItemList: Duplicate Records ---")
check_duplicate_rows(df_rewardsReceiptItems)
print("\n--- df_users: Duplicate Records ---")
check_duplicate_rows(df_users)

# 4. Referential Integrity

print("\n--- Referential Integrity: df_rewardsReceiptItems and df_receipts ---\n")
receipt_ids_items = set(df_rewardsReceiptItems['receipt_id'].unique())
receipt_ids_receipts = set(df_receipts['receipt_id'].unique())
print(f"Receipt IDs in rewarditems but not in receipts: {len(receipt_ids_items - receipt_ids_receipts)}")
print(f"Receipt IDs in receipts but not in rewarditems: {len(receipt_ids_receipts - receipt_ids_items)}")

print("\n--- Referential Integrity: df_users and df_receipts ---\n")
users_in_receipts = set(df_receipts['user_id'])
users_in_users = set(df_users['user_id'])
print(f"Users in receipts but not in users table: {len(users_in_receipts - users_in_users)}")
print(f"Users in users table but not in receipts: {len(users_in_users - users_in_receipts)}")

print("\n--- Referential Integrity: df_rewardsReceiptItems and df_brands ---\n")
brands_in_items = set(df_rewardsReceiptItems['brand_id'])
brands_in_brands = set(df_brands['brand_id'])
print(f"Brands in items but not in brands table: {len(brands_in_items - brands_in_brands)}")
print(f"Brands in brands table but not in items: {len(brands_in_brands - brands_in_items)}")




--- df_brands ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand_id      1167 non-null   object 
 1   barcode       1167 non-null   object 
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), object(7)
memory usage: 73.1+ KB
None

Missing values:
 brand_id          0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

Dumped values:
 topBrand     524
brandCode     35
dtype: int64

--- df_receipts ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
 #   